### Building NLP model using BERT

A project of Coursera for "Sentiment Analysis with Deep Learning Using BERT"

Dataset available on Kaggle: https://www.kaggle.com/tianysun/smilenotation/version/1

### EDA & Preprocessing

In [49]:
# Importing libraries
# pip install torch
import torch
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [4]:
df = pd.read_csv('smile-annotations-final.csv',
                names = ['id', 'text', 'category'])
df.set_index('id', inplace=True)

In [5]:
df.head()

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [6]:
df.category.unique()

array(['nocode', 'happy', 'not-relevant', 'angry', 'disgust|angry',
       'disgust', 'happy|surprise', 'sad', 'surprise', 'happy|sad',
       'sad|disgust', 'sad|angry', 'sad|disgust|angry'], dtype=object)

In [15]:
df[(df.category =='happy|sad')]

,text,category
id,,
612197303911346177,saw Indigenous Australia exhibition @britishmu...,happy|sad
611619022472355840,@britishmuseum interesting #periscope broadcas...,happy|sad
612981553929580544,"Sad to see @kettlesyard closed, but knowing it...",happy|sad
612003002547834880,Spent the evening @britishmuseum &amp; couldn'...,happy|sad
612993110147211264,@jennarbrown1 @britishmuseum Beautiful exhibit...,happy|sad
615456583829618688,@NationalGallery an inspiration to many a girl...,happy|sad
614720979089117184,"We enjoyed the exhibition, but we'd have liked...",happy|sad
615477231742251008,Looking forward to seeing #GlennLigon @tateliv...,happy|sad
615099713189249024,Queuing to get into @britishmuseum sucks but a...,happy|sad


In [14]:
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|disgust             2
sad|angry               2
sad|disgust|angry       1
Name: category, dtype: int64

In [16]:
df = df[~df.category.str.contains('\|')]

In [17]:
df = df[df.category != 'nocode']

In [18]:
df.category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [20]:
label = df.category.unique()
label_dict = {}
for i, l in enumerate(label):
    label_dict[l] = i
label_dict

{'happy': 0,
 'not-relevant': 1,
 'angry': 2,
 'disgust': 3,
 'sad': 4,
 'surprise': 5}

In [22]:
df["label"] = df.category.replace(label_dict)
df.head()

,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0


### Training - Validation Split
using stratify method to deal with class imbalance

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
x_train, x_val, y_train, y_val = train_test_split(df.index.values,
                                                 df.label.values,
                                                 test_size=0.15,
                                                 random_state=17,
                                                 stratify=df.label.values)

In [27]:
df['data_type'] = ['not_set']*df.shape[0]
df.loc[x_train, 'data_type'] = 'train'
df.loc[x_val, 'data_type'] = 'val'

In [28]:
df.groupby(['category', 'label', 'data_type']).count()

text
category     label data_type      
angry        2     train        48
                   val           9
disgust      3     train         5
                   val           1
happy        0     train       966
                   val         171
not-relevant 1     train       182
                   val          32
sad          4     train        27
                   val           5
surprise     5     train        30
                   val           5

### Encoding data using Tokenizer with BERT

In [30]:
# pip install transformers
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

/Users/rfdanti/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/rfdanti/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/rfdanti/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/rfdanti/anaconda3/lib/python3.7/site-packages/tensorflow/python/frame

In [31]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                         do_lower_case = True)

In [32]:
# Encoding data using tokenizer
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt')

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt')

In [33]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

In [34]:
dataset_train = TensorDataset(input_ids_train, 
                              attention_masks_train,
                             labels_train)
dataset_val = TensorDataset(input_ids_val, 
                              attention_masks_val,
                             labels_val)

In [35]:
print(len(dataset_train))
print(len(dataset_val))

1258
223


### Setting up BERT pretrained model & creating data loaders

In [37]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(label_dict),
    output_attentions = False,
    output_hidden_states=False)

In [39]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [40]:
# batch_size = 4 or 32

dataloader_train = DataLoader(dataset_train,
                             sampler=RandomSampler(dataset_train),
                             batch_size=4)

dataloader_val = DataLoader(dataset_val,
                             sampler=RandomSampler(dataset_val),
                             batch_size=32)

### Setting up Optimizer and Scheduler

In [41]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [42]:
optimizer = AdamW(model.parameters(), lr=1e-5, #2e-5 > 5e5
                 eps=1e-8)

In [77]:
epochs = 2

scheduler=get_linear_schedule_with_warmup(optimizer,
                                         num_warmup_steps=0,
                                         num_training_steps=len(dataloader_train)*epochs)

### Defining Performance Metrics

In [78]:
from sklearn.metrics import f1_score

In [79]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [93]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v:k for k, v in label_dict.items()}
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
            y_preds = preds_flat[labels_flat == label]
            y_true = labels_flat[labels_flat == label]
            print(f'Class: {label_dict_inverse[label]}')
            print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}')

### Creating Training Loop

In [81]:
import random


seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [82]:
## Only checking if cuda is available to use GPU enabled machine like google colab or GCP instead of local CPU

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cpu


Use Google Colab with its offer of free GPUs and TPUs, which can be added by going to the menu and selecting: Edit -> Notebook Settings -> Add accelerator (GPU).

Google Colab -- GPU Instance (K80)

- batch size = 32
- epoch = 10

In [83]:
# create function of model evaluation
# evaluation is like training but doesn't do backpropagation
# return loss average

def evaluate(dataloader_val):
    model.eval()
    
    loss_val_total = 0
    
    predictions, true_vals = [], []
    for batch in tqdm(dataloader_val):
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids': batch[0],
                 'attention_mask': batch[1],
                 'labels': batch[2]}
        
        with torch.no_grad():
            outputs = model(**inputs)
            
            loss = outputs[0]
            logits = outputs[1]
            loss_val_total += loss.item()
            
            logits = logits.detach().cpu().numpy()
            label_ids = inputs['labels'].cpu().numpy()
            predictions.append(logits)
            true_vals.append(label_ids)
            
        loss_val_avg = loss_val_total/len(dataloader_val)
        
        predictions = np.concatenate(predictions, axis=0)
        true_vals = np.concatenate(true_vals, axis=0)
        
        return loss_val_avg, predictions, true_vals

In [85]:
# pip install torch==1.4.0 nn_utils

for epoch in tqdm(range(1, epochs+1)): #iterate epoch
    #transform model into training mode
    model.train()
    #set training loss to 0 
    # then add each batch loss into this var (average training loss of each epoch)
    loss_train_total = 0
    
    #progress bar to see progress of training of each epoch
    progress_bar = tqdm(dataloader_train, 
                        desc='Epoch {:1d}'.format(epoch),
                        leave=False, disable=False)
    for batch in progress_bar:
        
        model.zero_grad()
        
        # 3 variables of batch loader
        # making sure each item is in the right device (important if using cuda)
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids' : batch[0],
                 'attention_mask' : batch[1],
                 'labels' : batch[2]}
        
        outputs = model(**inputs)
        
        loss = outputs[0]
        # add up the loss
        loss_train_total += loss.item()
        loss.backward()
        
        #normalised weight per layer
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        scheduler.step()
        
        #update progress bar to see loss per batch
        progress_bar.set_postfix(
            {'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
        
    # save model every epoch
    torch.save(model.state_dict(), f'bert_ft_epoch{epoch}.model')
    tqdm.write('\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write('Training loss: {loss_train_avg}')
    
    # call evaluation function to evaluate overtraining
    # it's when training loss decrease but validation loss increase
    # it means it no longer has generalisation ability
    # the model start to replicate training
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 score (weighted): {val_f1}')
        


Epoch {epoch}
Training loss: {loss_train_avg}


Validation loss: 0.2158374616077968
F1 score (weighted): 0.7564655172413793



Epoch {epoch}
Training loss: {loss_train_avg}


Validation loss: 0.22870748383658274
F1 score (weighted): 0.5288461538461537


### Loading and evaluating model

In [55]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = len(label_dict),
    output_attentions=False,
    output_hidden_states=False)

In [86]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [87]:
model.load_state_dict(torch.load('bert_ft_epoch1.model',
                                map_location=torch.device('cpu')))

<All keys matched successfully>

In [100]:
model.load_state_dict(torch.load('bert_ft_epoch2.model',
                                map_location=torch.device('cpu')))

<All keys matched successfully>

In [101]:
# we're not comparing loss anymore, 
#so only need to generate prediction and true values
_, predictions, true_vals = evaluate(dataloader_val)
# calculating for all batches

In [90]:
label_dict

{'happy': 0,
 'not-relevant': 1,
 'angry': 2,
 'disgust': 3,
 'sad': 4,
 'surprise': 5}

In [102]:
# evaluating prediction
accuracy_per_class(predictions, true_vals)

Class: happy
Accuracy: 23/23
Class: not-relevant
Accuracy: 0/4
Class: angry
Accuracy: 0/3
Class: sad
Accuracy: 0/1
Class: surprise
Accuracy: 0/1


The model failed to predict minority class, even though we used stratify technique in splitting train-test.
We can either try using more epochs, 
or even better: train with richer data with more class representation to train from.

In [99]:
import matplotlib.pyplot as plt
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    import itertools
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()